In [1]:
%matplotlib inline
import os
import numpy as np
import agent
import skimage
import skimage.io
import skimage.transform
from matplotlib import pyplot as plt
import random
import tensorflow as tf
import sys
sys.path.append('tensorflow-vgg/')
import vgg16

data_dir = '/home/vagrant/ocm/language-learning/data'

os.chdir('/home/vagrant/ocm/language-learning/code')

In [2]:
num_words = 2
vocab = ['W'+str(i) for i in range(num_words)]
embed_dim = 2
print(vocab)

['W0', 'W1']


In [3]:
logs_path = os.path.join('..','logs')

writer = tf.train.SummaryWriter(logs_path, graph=tf.get_default_graph())
print(logs_path)

Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.
../logs


In [4]:
def load_image(path):
    # load image
    img = skimage.io.imread(path)
    img = img / 255.0
    assert (0 <= img).all() and (img <= 1.0).all()
    # print "Original Image Shape: ", img.shape
    # we crop image from center
    short_edge = min(img.shape[:2])
    yy = int((img.shape[0] - short_edge) / 2)
    xx = int((img.shape[1] - short_edge) / 2)
    crop_img = img[yy: yy + short_edge, xx: xx + short_edge]
    # resize to 224, 224
    resized_img = skimage.transform.resize(crop_img, (224, 224))
    return resized_img.astype(np.float32)

In [5]:
#sender.show_images(target, distractor)

iterations = 1000

img_dirs = ['cat', 'dog']

os.getcwd()

'/home/vagrant/ocm/language-learning/code'

In [6]:
def get_image_activations(sess, vgg, image, placeholder):
    #image_pl = tf.placeholder("float32", [1, 224, 224, 3])
    batch = image.reshape((1, 224, 224, 3))
    feed_dict = {placeholder: batch}
    
    with tf.name_scope("content_vgg"):
        fc8 = sess.run(vgg.fc8, feed_dict=feed_dict)
    
    return(fc8)


In [7]:
tf.reset_default_graph()

sender = agent.SenderAgent(vocab)
reciever = agent.RecieverAgent(vocab)

## Run the iterations of the game
iterations = 10
num_classes = len(img_dirs)

wins = 0
losses = 0

update_estimators_every = 50

with tf.Session(config=tf.ConfigProto(gpu_options=(tf.GPUOptions(per_process_gpu_memory_fraction=0.7)))) as sess:
    vgg = vgg16.Vgg16()
    
    image_pl = tf.placeholder("float32", [1, 224, 224, 3])
    vgg.build(image_pl)
    sess.run(tf.global_variables_initializer())
    for i in range(iterations):
        im1_class, im2_class = np.random.choice(list(range(num_classes)), 2, replace=False)
        im1_dir, im2_dir = img_dirs[im1_class], img_dirs[im2_class]
        im1_path, im2_path = os.path.join(data_dir, 'images', im1_dir), os.path.join(data_dir, 'images', im2_dir)
        #print(im1_path, im2_path)

            # select random image in dirs
        im1_files, im2_files = os.listdir(im1_path), os.listdir(im2_path)
        im1_files = [i for i in im1_files if i.endswith('.jpg')]
        im2_files = [i for i in im2_files if i.endswith('.jpg')]

        im1 = np.random.choice(len(im1_files), 1)[0]
        im2 = np.random.choice(len(im2_files), 1)[0]

        target_file = os.path.join(im1_path, im1_files[im1])
        distractor_file = os.path.join(im2_path, im2_files[im2])

        # Load selected image
        target_image = load_image(target_file)
        distractor_image = load_image(distractor_file)
        
        target_acts = get_image_activations(sess, vgg, target_image, image_pl)
        distractor_acts = get_image_activations(sess, vgg, distractor_image, image_pl)
        
        comm_word = sender.show_images(sess, target_acts, distractor_acts)

        reordering = np.array([0,1])
        random.shuffle(reordering)
        target_ind = np.where(reordering==0)[0]
        print(reordering, target_ind)

        img_array = [target_acts, distractor_acts]
        i1, i2 = [img_array[reordering[i]] for i, img in enumerate(img_array)]
        selection = reciever.show_images(sess, comm_word, i1, i2) 

        if target_ind == selection:
            wins += 1
        else:
            losses += 1

    print(wins, losses)    

build model finished: 0s
[0 1] [0]
shape of scores (2, 1)
selected image  1 [[-20.28510094]
 [ 22.72819519]]
[0 1] [0]
shape of scores (2, 1)
selected image  0 [[ 14.16471481]
 [-12.52305126]]
[0 1] [0]
shape of scores (2, 1)
selected image  0 [[-13.86906719]
 [-14.60748672]]
[0 1] [0]
shape of scores (2, 1)
selected image  1 [[ -3.32337523]
 [ 23.53544044]]
[0 1] [0]
shape of scores (2, 1)
selected image  1 [[-24.91762543]
 [-13.86906719]]
[1 0] [1]
shape of scores (2, 1)
selected image  0 [[ 17.80216599]
 [-14.29872036]]
[0 1] [0]
shape of scores (2, 1)
selected image  1 [[ -7.53502464]
 [ 13.09079361]]
[1 0] [1]
shape of scores (2, 1)
selected image  1 [[-7.53502464]
 [-7.4211812 ]]
[1 0] [1]
shape of scores (2, 1)
selected image  1 [[-25.16317749]
 [-13.70547199]]
[1 0] [1]
shape of scores (2, 1)
selected image  0 [[-13.70547199]
 [-14.29872036]]
4 6
